# **Pixray Panorama demo**

Special thanks to [@dribnet's pixray repo](https://github.com/dribnet/pixray)
and [@altsoph](http://altsoph.com/)


In [ ]:
#@title **Setup**

#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. For setup,
#@markdown **you need to run this cell,
#@markdown then choose Runtime -> Restart Runtime from the menu,
#@markdown and then run the cell again**. It should remind you to
#@markdown do this after the first run.

#@markdown Setup can take 5-10 minutes, but once it is complete it usually does not need to be repeated
#@markdown until you close the window.

#@markdown **Note**: This installs the software on the Colab 
#@markdown notebook in the cloud and not on your computer.

# https://stackoverflow.com/a/56727659/1010653

# Add a gpu check
# (this can get better over time)
from google.colab import output

nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
  warning_string = f"--> GPU check: ONLY {gpu_memory} MiB available: WARNING, THIS IS PROBABLY NOT ENOUGH <--"
  print(warning_string)
  output.eval_js('alert("Warning - low GPU (see message)")')
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

from IPython.utils import io
with io.capture_output() as captured:
  # On 2021/10/08, Colab updated its default PyTorch installation to a version that causes
  # problems with diffvg. So, first thing, let's roll back to the older version:
  !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch/ -f https://download.pytorch.org/whl/torchvision/

  !git clone https://github.com/openai/CLIP
  # !pip install taming-transformers
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf pixray
  !git clone https://github.com/dribnet/pixray
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  # ClipDraw deps
  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom

  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  # !ls
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..

output.clear()
import sys
import shutil
import numpy as np
from PIL import ImageFile, Image, PngImagePlugin
sys.path.append("pixray")

def make_seed_img(from_fn, to_fn='cur_seed.png', delete_pixels = 20, shift_pixels = 240):
    in_img = Image.open(from_fn)
    seed_img_array = np.random.rand(in_img.size[1],in_img.size[0],3) * 255
    seed_img = Image.fromarray(seed_img_array.astype('uint8')).convert('RGB')
    # mask_img = Image.new(mode="RGB", size=(s_x, s_y), color=(0, 0, 0))
    seed_img.paste(in_img.crop((shift_pixels,0,in_img.size[0]-delete_pixels,in_img.size[1])), (0,0))
    seed_img.save(to_fn)
    # seed_img.crop((shift_pixels,0,seed_img.size[0]-delete_pixels,seed_img.size[1]))

result_msg = "setup complete"
import IPython
import os
if not os.path.isfile("first_init_complete"):
  # put stuff in here that should only happen once
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@title **First Frame Settings**

#@markdown Enter a description of what you want to draw - I usually add #pixelart to the prompt.
#@markdown If PixelDraw is not used, it will use VQGAN instead.
#@markdown <br>

prompts = "Scary Monsters And Nice Sprites. #pixelart #8bit" #@param {type:"string"}

aspect = "widescreen" ##param ["widescreen", "square"]

do_pixel = True #@param {type:"boolean"}

#@markdown Specify the desired palette ("" for default), here's a few examples:
#@markdown * red     (16 color black to red ramp)
#@markdown * rust\8  (8 color black to rust ramp)
#@markdown * black->red->white (16 color black/red/white ramp)
#@markdown * [#000000, #ff0000, #ffff00, #000080] (four colors)
#@markdown * red->yellow;[black]     (16 colors from ramp and also black)
#@markdown * Named colors can be anything in <a target=”_blank” href="https://xkcd.com/color/rgb/">this lookup table</a>

use_palette = "[#000000, #071008, #0e2011, #153019, #1c4022, #23502a, #2a6033, #31703b, #388044, #3f8f4c, #469f54, #4daf5d, #54bf65, #5bcf6e, #62df76, #69ef7f];black->white" #@param {type:"string"}
#@markdown Use this flag to encourage smoothess:
smoothness = True #@param {type:"boolean"} 

#@markdown Use this flag to encourage color saturation (use it against color fading):
saturation = True #@param {type:"boolean"} 

#@markdown When you have the settings you want, press the play button on the left.
#@markdown The system will save these and start generating images below.

#@markdown When that is done you can change these
#@markdown settings and see if you get different results. Or if you get
#@markdown impatient, just select "Runtime -> Interrupt Execution".
#@markdown Note that the first time you run it may take a bit longer
#@markdown as nessary files are downloaded.


#@markdown
#@markdown  **Advanced: you can also edit this cell and add add additional settings, combining settings from different notebooks.**



# Simple setup
import pixray

# these are good settings for pixeldraw
pixray.reset_settings()
pixray.add_settings(prompts=prompts, aspect=aspect)
pixray.add_settings(quality="better", scale=2.5)
pixray.add_settings(display_clear=True)

if do_pixel:
  pixray.add_settings(drawer="pixel")

# palette = None
if use_palette and use_palette!='None':
  pixray.add_settings(target_palette=use_palette)

if smoothness and smoothness!='None':
  pixray.add_settings(smoothness=2.0, smoothness_type='log')

if saturation:
  pixray.add_settings(saturation=1.0)

pixray.add_settings(noise_prompt_seeds=[1,2,3])

#### YOU CAN ADD YOUR OWN CUSTOM SETTING HERE ####
# this is the example of how to run longer with less frequent display
# pixray.add_settings(iterations=500, display_every=50)

settings = pixray.apply_settings()
pixray.do_init(settings)
pixray.do_run(settings)

shutil.copy('./output.png', './very_first_frame.png')

In [ ]:
#@title **Make sure you like the first frame, then start this cell to generate all other frames:**

shutil.copy('./very_first_frame.png', f'./frame_{0:03d}.png')

delete_from_right_pixels = 20 #@param
shift_pixels = 240 #@param
half_frames2generate = 5 #@param


for frame in range(half_frames2generate):
    shutil.copy('./output.png', f'./frame_{frame:03d}.png')
    make_seed_img(f'./frame_{frame:03d}.png')
    pixray.reset_settings()
    pixray.add_settings(prompts=prompts, aspect=aspect)
    pixray.add_settings(quality="better", scale=2.5)
    pixray.add_settings(display_clear=True)
    pixray.add_settings(init_image='./cur_seed.png')
    if do_pixel:
      pixray.add_settings(drawer="pixel")
    # palette = None
    if use_palette and use_palette!='None':
      pixray.add_settings(target_palette=use_palette)
    if smoothness and smoothness!='None':
      pixray.add_settings(smoothness=2.0, smoothness_type='log')
    if saturation:
      pixray.add_settings(saturation=1.0)
    pixray.add_settings(noise_prompt_seeds=[1,2,3])  

    settings = pixray.apply_settings()
    pixray.do_init(settings)
    pixray.do_run(settings)

    # shutil.copy('./output.png', f'./frame_{frame+1:03d}.png')


In [ ]:
#@title **Blend frames into the single image**

from glob import glob
# these values are okay for the widescreen aspect only
sidex = 500
sidey = 280


frames = []
for fn in glob('frame_*.png'):
    frames.append( fn )

pano_img = Image.new(mode="RGB", 
                     size=(sidex+shift_pixels*len(frames)-(sidex-shift_pixels), 
                           sidey+0*len(frames)), color=(255, 255, 255))

for idx, fr in enumerate(frames):
    fr_img = Image.open(fr)
    fr_img = fr_img.convert('RGB')
    if not idx:
        pano_img.paste( fr_img , (idx*shift_pixels, 0))
    else:
        pano_img.paste( fr_img.crop((shift_pixels,0,sidex,sidey)), ((idx+1)*shift_pixels, 0))
        for x in range(shift_pixels):
            w = x/shift_pixels
            for y in range(sidey):
                int_c = np.array(pano_img.getpixel((idx*shift_pixels+x,y)))*(1-w)+\
                        np.array( fr_img.getpixel((x,y)) ) *(w)
                int_c = int_c.astype(int).tolist()
                pano_img.putpixel((idx*shift_pixels+x,y),tuple(int_c))
pano_img.save('pano.png')
pano_img

In [ ]:
#@title **Compile and view the resulting video**

from tqdm.notebook import tqdm
from subprocess import Popen, PIPE

out_width = 500 #@param
speed = 3 #@param
fps = 25 #@param

pano_img =  Image.open('pano.png')
pano_img.size


frames = []
for idx, x in enumerate( range(0,pano_img.size[0]-out_width,speed) ):
    frames.append( pano_img.crop( (x,0,x+out_width,pano_img.size[1]) ) )


p = Popen(['ffmpeg',
           '-y',
           '-f', 'image2pipe',
           '-vcodec', 'png',
           '-r', str(fps),
           '-i',
           '-',
           '-vcodec', 'libx264',
           '-r', str(fps),
           '-pix_fmt', 'yuv420p',
           '-crf', '17',
           '-preset', 'veryslow',
           'pano.mp4'], stdin=PIPE)

for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()
p.wait()

from IPython.display import HTML
from base64 import b64encode
mp4 = open('pano.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)